In [1]:
%pip install 'vanna[chromadb,openai]'

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 584 kB 146 kB/s eta 0:00:01
  Using cached overrides-7.7.0-py3-none-any.whl (17 kB)
     |████████████████████████████████| 184 kB 89 kB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 30 kB/s eta 0:00:01
     |████████████████████████████████| 62 kB 27 kB/s eta 0:00:01
     |████████████████████████████████| 47 kB 93 kB/s eta 0:00:01
     |████████████████████████████████| 67 kB 52 kB/s eta 0:00:01
  distutils: /private/var/folders/6m/hp641khx64b7825bqjx6px100000gn/T/pip-build-env-twzkftqs/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/6m/hp641khx64b7825bqjx6px100000gn/T/pip-build-env-twzkftqs/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/6m/hp641khx64b7825bqjx6px100000gn/T/pip-

In [2]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore

/Users/borhandarvishi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:



class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': 'sk-proj-', 'model': 'gpt-4o-mini'})


In [9]:
vn.connect_to_sqlite('film.db')

In [10]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

Adding ddl: CREATE TABLE actor (
  actor_id numeric NOT NULL ,
  first_name VARCHAR(45) NOT NULL,
  last_name VARCHAR(45) NOT NULL,
  last_update TIMESTAMP NOT NULL,
  PRIMARY KEY  (actor_id)
  )


/Users/borhandarvishi/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [13:21<00:00, 104kiB/s] 


Adding ddl: CREATE INDEX idx_actor_last_name ON actor(last_name)

Adding ddl: CREATE TRIGGER actor_trigger_ai AFTER INSERT ON actor
 BEGIN
  UPDATE actor SET last_update = DATETIME('NOW')  WHERE rowid = new.rowid;
 END
Adding ddl: CREATE TRIGGER actor_trigger_au AFTER UPDATE ON actor
 BEGIN
  UPDATE actor SET last_update = DATETIME('NOW')  WHERE rowid = new.rowid;
 END
Adding ddl: CREATE TABLE country (
  country_id SMALLINT NOT NULL,
  country VARCHAR(50) NOT NULL,
  last_update TIMESTAMP,
  PRIMARY KEY  (country_id)
)
Adding ddl: CREATE TRIGGER country_trigger_ai AFTER INSERT ON country
 BEGIN
  UPDATE country SET last_update = DATETIME('NOW')  WHERE rowid = new.rowid;
 END
Adding ddl: CREATE TRIGGER country_trigger_au AFTER UPDATE ON country
 BEGIN
  UPDATE country SET last_update = DATETIME('NOW')  WHERE rowid = new.rowid;
 END
Adding ddl: CREATE TABLE city (
  city_id int NOT NULL,
  city VARCHAR(50) NOT NULL,
  country_id SMALLINT NOT NULL,
  last_update TIMESTAMP NOT NULL,
  PRI

In [11]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
CREATE TABLE category (
  category_id SMALLINT NOT NULL,
  name VARCHAR(25) NOT NULL,
  last_update TIMESTAMP NOT NULL,
  PRIMARY KEY  (category_id)
    )
""")

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="The tables of the database include film, film_category, actor, customer, rental, payment and inventory among others.")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="select release_year,count(*) from film group by release_year")


Adding ddl: 
CREATE TABLE category (
  category_id SMALLINT NOT NULL,
  name VARCHAR(25) NOT NULL,
  last_update TIMESTAMP NOT NULL,
  PRIMARY KEY  (category_id)
    )

Adding documentation....
Using model gpt-4o-mini for 68.5 tokens (approx)
Question generated with sql: How many films were released each year? 
Adding SQL...


'ec363b14-ccc6-5405-9bf1-5dcdeb7ecba3-sql'

In [15]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data.head(40)

,id,question,content,training_data_type
0,76c2e7b6-3bc9-575e-8818-5293ae71df84-sql,تعداد فیلم های ساخته شده بر اساس سال ساخت؟,"select release_year,count(*) from film group b...",sql
1,ec363b14-ccc6-5405-9bf1-5dcdeb7ecba3-sql,How many films were released each year?,"select release_year,count(*) from film group b...",sql
0,04151371-3217-53f4-a8e7-48f23f455f51-ddl,None,CREATE TRIGGER film_trigger_ai AFTER INSERT ON...,ddl
1,08a7614e-658e-51f2-8bcf-064e6131470d-ddl,None,CREATE TRIGGER address_trigger_au AFTER UPDATE...,ddl
2,0cb2d30d-6a8c-5a36-ac84-b5d8cd109135-ddl,None,CREATE TABLE actor (\n actor_id numeric NOT N...,ddl
3,0ed8a9b1-e9e7-590f-88f4-01f6255f3a69-ddl,None,CREATE INDEX idx_customer_fk_address_id ON cus...,ddl
4,113bb02a-4f1e-52fb-8769-d61f62c63cfa-ddl,None,CREATE TRIGGER film_actor_trigger_ai AFTER INS...,ddl
5,17c0afb5-69fb-52e3-b83f-11fcd042779b-ddl,None,CREATE TABLE language (\n language_id SMALLIN...,ddl
6,1a800738-2a86-5b30-a12d-b2b8c780ddd2-ddl,None,CREATE TRIGGER payment_trigger_ai AFTER INSERT...,ddl
7,1d0d3a9f-ea16-5c3a-a854-8ee7616c7286-ddl,None,CREATE INDEX idx_fk_film_category_film ON film...,ddl


<!-- # You can remove training data if there's obsolete/incorrect information. 
vn.remove_training_data(id='1-ddl') -->

In [ ]:
vn.ask(question=...)

In [14]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


Using model gpt-4o-mini for 902.25 tokens (approx)


Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


Using model gpt-4o-mini for 1018.75 tokens (approx)
Using model gpt-4o-mini for 115.5 tokens (approx)


...

Using model gpt-4o-mini for 1025.0 tokens (approx)
Using model gpt-4o-mini for 120.25 tokens (approx)


Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


Using model gpt-4o-mini for 1025.0 tokens (approx)


Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


Using model gpt-4o-mini for 705.0 tokens (approx)
